<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hu_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9p_9ebv8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-9p_9ebv8
  Resolved https://github.com/huggingface/transformers to commit 25b0f2033ba23e354ef2f665764248fcbb3f49ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [14]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [15]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [17]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hu", split="test")
#column_name = "audio, normalized_text"

In [18]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("tiny")

In [20]:
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 100MiB/s]


In [21]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [22]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hu")
    return result["text"]

In [23]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [24]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [25]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


 # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.3333333333333333, Precision: 0.7058823529411765, Recall: 0.8, F1: 0.7500000000000001
Processed 2 files, WER: 0.3181818181818182, Precision: 0.6666666666666666, Recall: 0.7, F1: 0.6829268292682926
Processed 3 files, WER: 0.47368421052631576, Precision: 0.5277777777777778, Recall: 0.5757575757575758, F1: 0.5507246376811594
Processed 4 files, WER: 0.391304347826087, Precision: 0.65, Recall: 0.5909090909090909, F1: 0.6190476190476191
Processed 5 files, WER: 0.4117647058823529, Precision: 0.6111111111111112, Recall: 0.6470588235294118, F1: 0.6285714285714287
Processed 6 files, WER: 0.4166666666666667, Precision: 0.5882352941176471, Recall: 0.625, F1: 0.6060606060606061
Processed 7 files, WER: 0.4375, Precision: 0.625, Recall: 0.625, F1: 0.625
Processed 8 files, WER: 0.3235294117647059, Precision: 0.6666666666666666, Recall: 0.7407407407407407, F1: 0.7017543859649122
Processed 9 files, WER: 1.0188679245283019, Precision: 0.07407407407407407, Recall: 0.1, F1: 0.08510